# Heavy, Medium и Light зрители
Пример формирования трех целевых аудиторий по объему просмотра и их использование в расчете

## Описание задачи и условий расчета
- Регион: Россия 0+
- Период: 19-22 декабря 2022 года
- Временной интервал: 21:00-23:00
- ЦА: все 18+, смотревшие телеканал Россия 1 с 12 по 15 декабря 2022 года с 21:00 до 23:00, разделенные на три группы. 
- Место просмотра: все места (дом и дача)
- Каналы: все каналы проекта TV Index
- Статистики: Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import json
import time

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике телекомпаний получим id РОССИЯ 1
cats.get_tv_company(name=["РОССИЯ 1 (СЕТЕВОЕ ВЕЩАНИЕ)"])

## Отбор зрителей
Отберем респондентов на основе их телепросмотра, разделим на группы и сохраним в переменные. Для этого посчитаем специальный отчет Consumption Target.

### Формирование задания
На этом этапе зададим все характеристики аудитории: возраст 18+, смотрели канал Россия 1 с 12 по 15 декабря 2022 года с 21:00 до 23:00.

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. 
date_filter = [('2022-12-12', '2022-12-15')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал: с 21:00 до 23:00
time_filter = 'timeBand1 >= 210000 AND timeBand1 < 230000'

# Задаем ЦА: Все 18+
basedemo_filter = 'age >= 18'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы: Россия 1
company_filter = 'tvCompanyId IN (1859)'

# Фильтр программ
program_filter = None

# Фильтр блоков
break_filter = None

# Фильтр роликов
ad_filter = None

# Указываем срезы
slices = []

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Статистики в отчете Consumption Target представляют собой тип просмотра, по которому отбираются зрители:
- CommonWatchers: просто зрители, смотрели телеканал хотя бы 1 минуту за отчетный период
- CommonNotWatchers: не зрители, не смотрели канал за отчетный период
- NGroupResp: зрители сортируются по длительности просмотра и разбиваются на n равных по количеству респондентов групп
- NGroupDur: зрители сортируются по длительности просмотра и разбиваются на n групп по длительности. В этом случае количество респондентов в группах будет разным. 
        
В данном примере мы разобъем всех зрителей на группы равные по выборке:

In [ ]:
# Указываем список статистик для расчета
statistics = ["NGroupResp"]

Дополнительные настройки - consumption_target_options. Здесь можно указать:

- "groupsCount" - количество групп, на которое необходимо поделить отобранных респондентов. Работает в связке со статистиками NGroupResp и NGroupDur
- "intervalConditions" - длительность просмотра (в минутах). Можно указывать без конечной или начальной границы.
- "viewingType" - тип просмотра
    - "NONCONSECUTIVE" - непоследовательный просмотр, с переключениями
    - "CONSECUTIVE" - последовательный просмотр (без переключений)

Пример:

`
consumption_target_options = {
    "groupsCount": "5",
    "intervalConditions": {
        "intervals": [
            {
                "from":"5",
                "to": "100"
            }
        ],
    "viewingType": "CONSECUTIVE"
    }
}
`

Нам нужно разбить всех зрителей на три группы, поэтому передадим в параметр `groupsCount` значение 3:

In [ ]:
consumption_target_options = {
    "groupsCount": "3"
}

### Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_consumption_target_task(date_filter=date_filter, weekday_filter=weekday_filter,
                                                daytype_filter=daytype_filter, time_filter=time_filter,
                                                basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter, 
                                                location_filter=location_filter, company_filter=company_filter,
                                                program_filter=program_filter, break_filter=break_filter, ad_filter=ad_filter,
                                                slices=slices, statistics=statistics, options=options,
                                                consumption_target_options=consumption_target_options)

# Отправляем задание на расчет и ждем выполнения
task_consumption_target = mtask.wait_task(mtask.send_consumption_target_task(task_json))

# Получаем результат
df_cons = mtask.result2table(mtask.get_result(task_consumption_target))
df_cons

### Сохранение переменных

Для последующего использования в расчетах, сохраним полученные группы как переменные.  

Создадим словарь, где ключ - номер группы, значение - название переменной: 

In [ ]:
consumptions = {
    '1': 'All18+_R1_20221212_20221215_21-23_light',
    '2': 'All18+_R1_20221212_20221215_21-23_medium',
    '3': 'All18+_R1_20221212_20221215_21-23_heavy' 
}

Передадим каждую группу в метод сохрания переменной:

In [ ]:
for nGroup, var_name in consumptions.items():

    cats.add_custom_respondent_variable(
        resp = df_cons[df_cons['nGroup'] == nGroup], #датафрейм с посчитанной переменной
        name = var_name, #название переменной    
        is_public=False #Переменная будет доступна только для создавшего ее логина
    )
    
    time.sleep(2)

In [ ]:
# выведем список всех переменных
resp_vars = cats.get_custom_respondent_variable()
resp_vars

In [ ]:
# выведем созданные нами переменные
var_names = list(consumptions.values())
resp_vars[resp_vars['name'].isin(var_names)]

In [ ]:
# удаление переменной (раскоментируйте строки ниже для выполнения)
#var_id = resp_vars[resp_vars['name'] == "All18+_R1_20221212_20221215_21-23_light"]['id'].values[0] #получаем id переменной по ее имени
#cats.delete_custom_respondent_variable(var_id) #удаляем переменную из списка

## Расчет отчета по группам зрителей
Используем сохраненные переменную для расчета timeband отчета
### Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. 
date_filter = [('2022-12-19', '2022-12-22')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал: с 19:00 до 23:00
time_filter = 'timeBand1 >= 210000 AND timeBand1 < 230000'

# Задаем ЦА: оставляем незаполненным. Группу зрителей укажем далее в специальном параметре.
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = None

# Указываем список статистик для расчета
statistics = ['Share']

# Указываем срезы: телесеть
slices = ['tvNetName']

# Задаем условия сортировки: телесеть (от а до я)
sortings = {"tvNetName":"ASC"}

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
}

Сформируем словарь переменных, где ключ - название, которое будет отображаться в отчете, а значение - id переменной:

In [ ]:
# укажите здесь id, созданных вами переменных
variables = {
    'Россия 1 18+ 12-15 дек 2022 21-23 [light]': '545a7675-304e-42ac-aa5e-1becfa40be87',
    'Россия 1 18+ 12-15 дек 2022 21-23 [medium]': 'dad86153-7a8e-4be9-a256-6dafe7701f2d',
    'Россия 1 18+ 12-15 дек 2022 21-23 [heavy]': '38b8f92c-8c3b-4466-a22e-511b16ac2c40'
}

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой переменной формируем задание и отправляем на расчет
for var_name, var_id in variables.items():
    
    # Подставляем значения словаря в параметры
    project_name = var_name 
    custom_respondent_variable_id = var_id
              
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_timeband_task(task_name=project_name, date_filter=date_filter, 
                                          weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                          company_filter=company_filter, time_filter=time_filter, 
                                          basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                          location_filter=location_filter, slices=slices, 
                                          statistics=statistics, sortings=sortings, options=options,
                                          custom_respondent_variable_id=custom_respondent_variable_id)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')
    
print(f"\nid: {[i['task']['taskId'] for i in tasks]}")

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

# Приводим порядок столбцов в соответствие с условиями расчета
df = df[['prj_name']+slices+statistics]
df

## Настройка внешнего вида таблицы
Пропустите этот шаг, если хотите экспортировать таблицу в ее текущем виде

In [ ]:
# Формируем сводную таблицу: строки - интервалы; столбцы - типы дня; значения - статистики
df = pd.pivot_table(df, values=statistics,
                        index=['tvNetName'], 
                        columns=['prj_name'])
df

In [ ]:
# Поменять местами уровни в столбцах (раскомментируйте при необходимости)
#df = df.swaplevel(0,1, axis=1).sort_index(axis=1)

#df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('consumption_target_02_hml')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)